<a href="https://colab.research.google.com/github/BlueBicycleBlog/finding-data-demo-08-21/blob/master/SRA_Cloud_RNASeq-demo-Google_CoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo notebook for "Finding Data for your Research Organism: Plants and RNASeq Data"

This notebook was originally part of a webinar presentation to demonstrate a variety of NCBI tools: SRA toolkit, NCBI Datasets, MagicBlast and Genome Data Viewer.

A few changes have been made to make sure this notebook works in 2024, but for the most part a very good guide on this notebook can be found by watching the youtube video.

[Link to the Youtube Video](https://www.youtube.com/watch?v=BBBKt7Z3bt8)

![Youtube Video](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/youtube.png?raw=1)



A brief summary of the dataset from BioProjects at NCBI

![Bioproject Summary](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/bioproject.png?raw=1)


[A weblink with more details](https://www.ncbi.nlm.nih.gov/bioproject?term=320374[top+bioproject]+NOT+320378[uid])

### How to Run this Notebook if Not in the Binder (e.g. Codespace or Google CoLab)

If you are not running in the binder, then some conda installs will need to be done  before using this notebook

For Google CoLab only, run this code cell.

It is okay if it says that the kernel has crashed -- it is supposed to restart.




In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


For Codespace AND Google CoLab, continue to install.


This will take a few minutes the first time.

In [28]:
#The first two lines are specific to the Google CoLab conda install
!sudo apt-get install libarchive13
!conda install mamba -c conda-forge --force-reinstall -y

#These install lines add the necessary packages.

!conda install -y conda-forge::ncbi-datasets-cli
!conda install -y bioconda::sra-tools
!conda install -y bioconda::magicblast
!conda install -y bioconda::blast
!sudo apt-get install -y samtools uuid-runtime


Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bioconda::magicblast


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-23.1.0               |  py310hff52083_0         918 KB  conda-forge
    conda-libmamba-solver-23.1.0|     pyhd8ed1ab_0          39 KB  conda-forge
    cryptography-43.0.1        |  py310h6c63255_0         1.4 MB  conda-forge
    fmt-9.1.0                  |       h924138e_0         185 KB  conda-forge
    icu-70.1                   |       h27087fc_0        13.5 MB  conda-forge
    importlib-metadata-8.5.0   |     pyha770c72_0          28 KB  conda-forge
    libarchive-3.5.2           |       hada088e_3         1.6 MB  conda-forge
    libmamba-1.1.0             |       h70b1f8a_2         1.4 MB  conda-forge
    libmambapy-1.1.0  

A few notes:

* This demo is intended to be run on binder as all programmatic installs have been done directly to the binder server
* If you wish to download and use this notebook on your own server, please find the instructions for the programmatic installs in the README.md file for this git repo: https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/README.md
* The commands in this notebook that begin with "!" are bash commands that can be copied and run directly in a terminal.

Command Line programs already loaded onto the server, but if you are running this notebook locally, you may need to install and add these programs to your path to work with these commands as written.

**SRA Toolkit** (install instructions here: https://github.com/ncbi/sra-tools/wiki/02.-Installing-SRA-Toolkit)

**Datasets Command Line** (install instructions here: https://www.ncbi.nlm.nih.gov/datasets/docs/quickstarts/command-line-tools/

**Blast+ Executables** (install instructions here: https://www.ncbi.nlm.nih.gov/books/NBK279690/)

**MagicBlast** (install instructions here: https://ncbi.github.io/magicblast/)

## Configure SRA Toolkit

If this is the first time running the notebook, use the following cell to set up SRA toolkit

In [2]:
GUID = !uuidgen
GUID = str(GUID).strip('[]')
!vdb-config --set LIBS/GUID={GUID}
!vdb-config --report-cloud-identity yes
!vdb-config --set /repository/user/main/public/cache-disabled=true
!cat $HOME/.ncbi/user-settings.mkfg

2024-09-26T16:50:20 vdb-config.3.1.1 err: condition violated while updating node - Warning: normally this application should not be run as root/superuser
Report Cloud Instance Identity was set to true
2024-09-26T16:50:20 vdb-config.3.1.1 err: condition violated while updating node - Warning: normally this application should not be run as root/superuser


Test that the sra toolkit is configured and working from this location by downloading the first two reads with quality scores using an SRA Accession.

In [3]:
!fastq-dump --stdout -X 2 SRR3241527

Read 2 spots for SRR3241527
Written 2 spots for SRR3241527
@SRR3241527.1 FCC3E8RACXX:1:1101:8441:2000# length=100
NCCACCCTTCGAAATCACAGTCCGTGAGTCTCCAACATTGGCAACATAAAGATGATCACCAACTAGCACTGCTGTTGAAGCAGTAGAACCATCATCACGG
+SRR3241527.1 FCC3E8RACXX:1:1101:8441:2000# length=100
#4BDFFFFHHHHHJJJIJJJHIIJGHIIEGHIJJJJJJJJIJJJJJJJJJIJJJJJJJJJJJJJJJHHHHHFFFFDFEEEEEDCCDDDDDDCDDDDDDDB
@SRR3241527.2 FCC3E8RACXX:1:1101:9310:1999# length=100
NGTGAGCATCCGCAGGGACCATCGCAATGCTTTGTTTTAATTAAACAGTCGGATTCCCCTTGTCCGTACCAGTTCTGAGTCGACCGTTCGACGCCCGGGG
+SRR3241527.2 FCC3E8RACXX:1:1101:9310:1999# length=100
#4=DDFFFHHHHHJIHJJJIJIJJJJJIIJJJJIIIJJJIJJJJJJJJFIIJJJJHHHHEFFFFFDDEDDDDCDDCDDDDDDDDDBDDDDDDDDDD@9<B


## Load sequence data using SRA Toolkit command fastq-dump

For convenience, the accession list that we generated using the SRA Run Selector has been added to this notebook environment for you. You can reuse this notebook to run your own accessions (memory permitting) by just replacing this text file in the directory with your own list.

These 12 accessions belong to a Bioproject that measured RNA expression in Populus trichocarpus xylem on days 0, 5 and 7 of a drought treatment.

![Metadata SRA](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/metadata.png?raw=1)

This metadata table was generated using the SRA Run Selector tool.

[SRA Run Selector](https://www.ncbi.nlm.nih.gov/Traces/study/?query_key=5&WebEnv=MCID_66f52a3b8c49702d712a7c8e&o=acc_s%3Aa)

In [6]:
!wget https://raw.githubusercontent.com/BlueBicycleBlog/finding-data-demo-08-21/refs/heads/master/accessionlist.txt

--2024-09-26 16:52:37--  https://raw.githubusercontent.com/BlueBicycleBlog/finding-data-demo-08-21/refs/heads/master/accessionlist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132 [text/plain]
Saving to: ‘accessionlist.txt’

accessionlist.txt   100%[===================>]     132  --.-KB/s    in 0s      

2024-09-26 16:52:37 (7.06 MB/s) - ‘accessionlist.txt’ saved [132/132]



In [7]:
!cat accessionlist.txt

SRR3241527
SRR3241528
SRR3241529
SRR3241530
SRR3241531
SRR3241532
SRR3241533
SRR3241534
SRR3241535
SRR3241536
SRR3241537
SRR3241538


In [8]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [9]:
!cat accessionlist.txt | xargs -P 12 -n 1 fastq-dump -X 1000000 -O data

Read 1000000 spots for SRR3241531
Written 1000000 spots for SRR3241531
Read 1000000 spots for SRR3241536
Written 1000000 spots for SRR3241536
Read 1000000 spots for SRR3241535
Written 1000000 spots for SRR3241535
Read 1000000 spots for SRR3241533
Written 1000000 spots for SRR3241533
Read 1000000 spots for SRR3241528
Written 1000000 spots for SRR3241528
Read 1000000 spots for SRR3241538
Written 1000000 spots for SRR3241538
Read 1000000 spots for SRR3241527
Written 1000000 spots for SRR3241527
Read 1000000 spots for SRR3241529
Written 1000000 spots for SRR3241529
Read 1000000 spots for SRR3241530
Written 1000000 spots for SRR3241530
Read 1000000 spots for SRR3241532
Written 1000000 spots for SRR3241532
Read 1000000 spots for SRR3241534
Written 1000000 spots for SRR3241534
Read 1000000 spots for SRR3241537
Written 1000000 spots for SRR3241537


In [10]:
!ls data

SRR3241527.fastq  SRR3241530.fastq  SRR3241533.fastq  SRR3241536.fastq
SRR3241528.fastq  SRR3241531.fastq  SRR3241534.fastq  SRR3241537.fastq
SRR3241529.fastq  SRR3241532.fastq  SRR3241535.fastq  SRR3241538.fastq


## Download the RefSeq Genome and annotations from Datasets

The NCBI Datasets datasets command line tools are **datasets** and **dataformat**.

Use datasets to download biological sequence data across all domains of life from NCBI.

Use dataformat to convert metadata from JSON Lines format to other formats.

Note: The NCBI Datasets command line tools are currently in alpha and will be updated frequently to add new features, fix bugs, and enhance usability. Command syntax is subject to frequent changes. Please check back often for updates.

This part of the analysis repliates what we did with the web browser, except by using the command line tool provided by NCBI.

In [ ]:
!datasets download genome accession GCF_000002775.4 --chromosomes 5 --include genome,gtf


In [12]:
!unzip -o ncbi_dataset.zip -d PtRefSeq

Archive:  ncbi_dataset.zip
  inflating: PtRefSeq/README.md      
  inflating: PtRefSeq/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna  
  inflating: PtRefSeq/ncbi_dataset/data/GCF_000002775.4/genomic.gtf  
  inflating: PtRefSeq/ncbi_dataset/data/dataset_catalog.json  
  inflating: PtRefSeq/md5sum.txt     


Magic-BLAST requires a BLAST database in order to align the reads.

In [13]:
!makeblastdb -in PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna -dbtype nucl -parse_seqids -out PtRefSeq/Pt.chrom5




Building a new DB, current time: 09/26/2024 16:55:35
New DB name:   /content/PtRefSeq/Pt.chrom5
New DB title:  PtRefSeq/ncbi_dataset/data/GCF_000002775.4/chr5.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 1 sequences in 1.04377 seconds.




## Align Fastq Files to Reference Genome to Produce Bams using Magic-BLAST

Magic-BLAST is a tool for mapping large next-generation RNA or DNA sequencing runs against a whole genome or transcriptome. Each alignment optimizes a composite score, taking into account simultaneously the two reads of a pair, and in case of RNA-seq, locating the candidate introns and adding up the score of all exons. This is very different from other versions of BLAST, where each exon is scored as a separate hit and read-pairing is ignored.

Create a location to store the bams.

In [14]:
!mkdir bams

This next command limits the number of bases that are input into the magicblast command before running it. The default is 50M. The only reason the value is being set lower is to allow this notebook to work on the binder server, which as a 2 GB virtual memory limit. If the limit is still too high when running the notebook, try reducing the BATCH_SIZE to 5000000.

In [15]:
%env BATCH_SIZE=7500000

env: BATCH_SIZE=7500000


Magic-BLAST has a variety of parameters that can be set. In this example, we are setting the parameters:

* *splice* is *True* (transcriptomic data)
* *no_unaligned* (to reduce the file size that is generated)
* *max_db_word_count* is 10 to reduce virtual memory demand (Magicblast rejects 16-base word matches between input reads and the target database if this word appears in the database more than <number> of times. The default is 30.

Samtools

The default output type for MagicBlast is a sam file, so we pipe it into samtools to make a bam for later viewing on the Genome Data Viewer. The sort command has a large virtual memory command.

* *-m* parameter is set to 250M to reduce load in this binder notebook (it can be removed on your own machine)\

The first example only runs magicblast on a single file.

To run the program on all of the files, then uncomment the advanced cell which takes in the entire list of fastq files and runs magicblast. This will take longer.


In [29]:
!magicblast -query data/SRR3241538.fastq -db PtRefSeq/Pt.chrom5 -splice T -no_unaligned -infmt fastq -max_db_word_count 10 | \
samtools view -bS | samtools sort -m 250M -o bams/SRR3241538.bam -O BAM

### Optional

Only uncomment and run this next code cell if you want to get all of the bams generated.

This is not advised on a smaller or temporary machine, so proceed with caution.

In [31]:
#!cat accessionlist.txt | xargs -I {} -P 12 -n 1 echo "magicblast -query data/{}.fastq -db PtRefSeq/Pt.chrom5 -splice T -no_unaligned -infmt fastq -max_db_word_count 10 | \
samtools view -bS | samtools sort -m 250M -o bams/{}.bam -O BAM" | while read line; do sh -c "$line"; done

^C


The bam files need to be indexed to be used in Genome Data Viewer.

In [33]:
!ls bams

SRR3241527.bam


In [34]:
!for i in bams/*.bam; do samtools index $i; done

In [35]:
!ls bams

SRR3241527.bam	SRR3241527.bam.bai


In [ ]:
#If you are using your own accession list, you can generate the list of command by running this bash command after removing the "#" from the beginning of each line

#!cat accessionlist.txt| while read line;do \
#echo "magicblast -query data/$line.fastq -db Pt.chrom5 -splice T -no_unaligned -infmt fastq -max_db_word_count 10 | \
#samtools view -bS | samtools sort -m 250M -o bams/$line.bam -O BAM"; done

In [ ]:
#If you wish to continue with analyzing the bam files in this notebook, here is a shortcut code.
#Just remove the # in front of the following two lines to run them.

#!cat accessionlist.txt | xargs -I % wget -O bams/%.bam https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/%.bam
#!cat accessionlist.txt | xargs -I % wget -O bams/%.bam.bai https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/%.bam.bai

A few of the tracks (one for Day 0, One for Day 5 and one for Day 7) have been entered and can be viewed by following this link which will be available until November 16, 2024 (90 days after creation).

https://www.ncbi.nlm.nih.gov/gdv/browser/genome/?cfg=NCID_1_24060339_130.14.22.10_9146_1727372312_2335023902


If the link has expired,

1.) Navigate to the GDV Viewer for Populus trichocarpus chromosome 5

![Navigate to Chromosome 5](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/navigate_GDV_Viewer.png?raw=1)

2.) Upon opening GDV Viewer, the bam files and their corresponding indexes can be loaded using the remote files that we have put into our ftp repository (GDV does not currently support local uploads of bam files).

Make sure to select the same version of the genome that was downloaded from datasets.

![Genome Version](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/genome_version.png?raw=1)

Loading all 12 files may make it difficult to view in one webpage, so we have chosen just three of the files form different treatment days for our demo.

One each from Day 0, Day 5, and Day 7.

These links can be used if you want more practice loading remote files into Genome Data Viewer.


https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241527.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241528.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241529.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241530.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241531.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241532.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241533.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241534.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241535.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241536.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241537.bam
https://ftp.ncbi.nlm.nih.gov/pub/education/public_webinars/2021/08Aug18_Res_Org_2/A_bams/SRR3241538.bam



![How to load remote files](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/add_remote_track.png?raw=1)




If you want to load your own remote files into GDV Viewer, it is possible using your Github repository for files that are less than 1 GB. Just use the following file format based on the location of your file in the repository.

This file has been added to this repository for testing this option.

https://github.com/BlueBicycleBlog/finding-data-demo-08-21/raw/master/SRR3241537.bam

Make sure the `bam` files and the `bam.bai` fils are in the same location in your repository.



Read More about the research project that produced this data set.

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE79401

Li S, Lin YJ, Wang P, Zhang B et al. The AREB1 Transcription Factor Influences Histone Acetylation to Regulate Drought Responses and Tolerance in <i>Populus trichocarpa</i>. Plant Cell 2019 Mar;31(3):663-686.

[PubMed Link to Publication](https://pubmed.ncbi.nlm.nih.gov/30538157/)

[Link to Publication where Supplemental Links Work](https://academic.oup.com/plcell/article/31/3/663/5985576?login=false)


One of the findings in the paper is that the ADA2 genes may confer drought tolerance

The main gene for ADA2 is not on the chromosome mapped here, but doing a searcher for "ADA2" in the Genome viewer brought us to an interesting set of coordinates in Chromosome 5.

![Possible Alternative Transcription in SRM1](https://github.com/BlueBicycleBlog/finding-data-demo-08-21/blob/master/img/alternative_transcription.png?raw=1)

The gene that is in the main screen of the genome viewer when clicking on the link provided is "SRM1", also known as Salt-Related MYB1.

This is a gene that can impact vegetation growth and leaf shape.

In this snapshot, looking at the first exon in the putative gene shows that a more complete version of the first exonic region appears to be active on Days 5 and 7 when compared to Day 0.

Try loading the other bam files to see if the replicates support this finding.

All the bam file links have been provided.





